# Fleming Pipeline - ipython notebook version

Note on astrometry.net keys:
[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Key shouldn't matter if not `force_upload==False`
~~Need to add your own key to `~/.astroquery/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`~~

In [ ]:
## Imports
import Reducer
import ShiftFinder
import FluxFinder
import DataAnalyser
import Constants
import Utilities
import MovingObjectFinder
import StreakFinder
import Cataloguer

from datetime import datetime
import os
import importlib

#from astroquery import log
#log.setLevel("TRACE")


In [ ]:
## Reload imports (if files changed, only used for testing)
Reducer = importlib.reload(Reducer)
ShiftFinder = importlib.reload(ShiftFinder)
FluxFinder = importlib.reload(FluxFinder)
DataAnalyser = importlib.reload(DataAnalyser)
Utilities = importlib.reload(Utilities)
Cataloguer = importlib.reload(Cataloguer)
Constants = importlib.reload(Constants)


In [ ]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
#COLOR = "white"
COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:
config = Constants.Config(
    #image_subdir = "../images/0121",
    image_dir = "/home/callum/mnt/data/jgtdata/l137_0/0121",
    image_prefix = "l137_0",
    bias_prefix = "bias_shutter",
    has_sets = True,
    set_size = 50,
    n_sets = 7
)

In [ ]:
r = Reducer.Reducer(config, "No filter") ## Only "No filter" for Trius
#r.reduce(skip_existing=True)
r.reduce(skip_existing=False)

In [ ]:
c = Cataloguer.Cataloguer(config)

catalogue_image = os.path.join(config.image_dir, config.image_format_str.format(1, 1))
c.catalogue(catalogue_image, solve=False)

In [ ]:
sf = ShiftFinder.ShiftFinder(config)
sf.get_all_shifts()

In [ ]:
ff = FluxFinder.FluxFinder(config, c.get_n_sources())

#ff.find_all_fluxes()
ff.make_light_curves()

In [ ]:
da = DataAnalyser.DataAnalyser(config)

print("[Main] Doing things for unadjusted curves")
da.get_means_and_stds(adjusted=False)
variable_ids = da.get_variables(ff, adjusted=False)
da.plot_means_and_stds(adjusted=False)

avg_ids = da.get_ids_for_avg()
ff.make_avg_curve(avg_ids)

print("[Main] Adjusting")
ff.divide_by_average()

In [ ]:
print("[Main] Plotting light curves")
avg_fname = "{}_avg{}".format(config.image_prefix, config.standard_file_extension)
avg_path = os.path.join(config.workspace_dir, avg_fname)
ff.plot_avg_light_curve(avg_path, adjusted=True, show=True)
ff.plot_all_light_curves(variable_ids, adjusted=True, show=True)

In [ ]:
da.get_means_and_stds(adjusted=True)
da.get_variable_ids(ff, adjusted=True)
da.plot_means_and_stds(adjusted=True)
da.output_results()
da.create_thumbnails(ff)

## Debug land

In [62]:
## Imports
import Reducer
import ShiftFinder
import FluxFinder
import DataAnalyser
import Constants
import Utilities
import MovingObjectFinder
import StreakFinder
import Cataloguer

from datetime import datetime
import os
import importlib

In [63]:
debug_config = Constants.Config(
    #image_subdir = "../images/0121",
    image_dir = "/home/callum/mnt/data/jgtdata/l137_0/0121",
    image_prefix = "l137_0",
    has_sets = True,
    set_size = 10,
    n_sets = 5
)

In [5]:
Reducer = importlib.reload(Reducer)
ShiftFinder = importlib.reload(ShiftFinder)
FluxFinder = importlib.reload(FluxFinder)
DataAnalyser = importlib.reload(DataAnalyser)
Utilities = importlib.reload(Utilities)
Cataloguer = importlib.reload(Cataloguer)
Constants = importlib.reload(Constants)

In [64]:
Cataloguer = importlib.reload(Cataloguer)

c = Cataloguer.Cataloguer(debug_config)

catalogue_image = os.path.join(debug_config.image_dir,
        debug_config.image_format_str.format(1, 1))

n_sources = c.generate_catalogue(catalogue_image, solve=False)
c.generate_image_times()

[Cataloguer] Filtered out 0 objects
[Cataloguer] Found 1224 suitable sources
[Cataloguer] Getting coordinate system for image '/home/callum/mnt/data/jgtdata/l137_0/0121/r_l137_0_1_001.fit'
[Cataloguer] Generating image time file
[DEBUG] looping images


In [68]:
ShiftFinder = importlib.reload(ShiftFinder)
sf = ShiftFinder.ShiftFinder(debug_config, n_sources)
sf.get_reference_coordinates(10)
sf.generate_shifts()

[DEBUG] looping images
[ShiftFinder] Finding shifts in image: set 1; image 001...


-1.6899800789202386 -0.0016169203648637165
[ShiftFinder] ...Shifts: -1.6899800789202386,-0.0016169203648637165
[ShiftFinder] Finding shifts in image: set 1; image 002...


0.739505117174625 -0.09066331872238464
[ShiftFinder] ...Shifts: -0.9504749617456136,-0.09228023908724836
[ShiftFinder] Finding shifts in image: set 1; image 003...


-2.8693208346784633 -0.07699147903235204
[ShiftFinder] ...Shifts: -2.1298157175038384,-0.16765479775473668
[ShiftFinder] Finding shifts in image: set 1; image 004...


4.068158565637768 0.8417925894825906
[ShiftFinder] ...Shifts: 1.1988377309593048,0.7648011104502386
[ShiftFinder] Finding shifts in image: set 1; image 005...


...

-1.3482054354374942 2.049813787574486
[ShiftFinder] ...Shifts: -0.6487143824639956,1.8446095598181493
[ShiftFinder] Finding shifts in image: set 5; image 007...


-0.33354617109842444 -0.7015139168654199
[ShiftFinder] ...Shifts: -1.6817516065359186,1.348299870709066
[ShiftFinder] Finding shifts in image: set 5; image 008...


-0.6761682857100118 -2.2299124535196397
[ShiftFinder] ...Shifts: -1.0097144568084362,-2.9314263703850596
[ShiftFinder] Finding shifts in image: set 5; image 009...


0.8530855714850825 -2.7302083200540443
[ShiftFinder] ...Shifts: 0.1769172857750707,-4.960120773573684
[ShiftFinder] Finding shifts in image: set 5; image 010...


1.2880962309774189 1.4386539140191914
[ShiftFinder] ...Shifts: 2.1411818024625013,-1.291554406034853


In [ ]:
FluxFinder = importlib.reload(FluxFinder)
config.n_sets = 1
config.set_size = 1
ff = FluxFinder.FluxFinder(config, c.get_n_sources())
ff.make_light_curves()

In [ ]:
DataAnalyser = importlib.reload(DataAnalyser)
da = DataAnalyser.DataAnalyser(config)
variable_ids = da.get_variable_ids()
print(variable_ids)